In [1]:
from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
%matplotlib inline

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/thiago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv('imdb-reviews-pt-br.csv', encoding='utf-8')

In [4]:
# mapeando as classes de resposta para valores numéricos
df["sentiment_int"] = df["sentiment"].map({"pos": 0, "neg": 1})
del df['text_en']
del df['sentiment']
df.head()

,id,text_pt,sentiment_int
0,1,"Mais uma vez, o Sr. Costner arrumou um filme p...",1
1,2,Este é um exemplo do motivo pelo qual a maiori...,1
2,3,"Primeiro de tudo eu odeio esses raps imbecis, ...",1
3,4,Nem mesmo os Beatles puderam escrever músicas ...,1
4,5,Filmes de fotos de latão não é uma palavra apr...,1


In [5]:
texto = df['text_pt']
classes = df['sentiment_int']

In [6]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [7]:
# Aplicando o stemming em nossa base:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

In [8]:
def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

In [9]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def Lemmatization(instancia):
  palavras = []
  for w in instancia.split():
    palavras.append(wordnet_lemmatizer.lemmatize(w))
  return (" ".join(palavras))

In [10]:
def Preprocessing(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','').replace('"','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [11]:
# Aplica a função em todos os dados:
texto = [Preprocessing(i) for i in texto]

In [12]:
texto[0]

'vez, sr costner arrumou filme tempo necessário além terríveis seqüências resgate mar, quais poucas, simplesmente importei nenhum personagens maioria fantasmas armário, personagem costers realizado logo início, esquecido tarde, importava personagem deveríamos importar arrogante superconfiante, ashton kutcher problema sai garoto pensa melhor qualquer outra pessoa redor mostra sinais armário desordenado único obstáculo parece estar vencendo costner finalmente, bem além meio caminho, costner conta sobre fantasmas kutchers informados kutcher levado ser melhor pressentimentos presságios anteriores nenhuma mágica aqui, tudo podia fazer desligar hora'

In [13]:
from nltk.tokenize import word_tokenize as tokenizador

In [14]:
# Instancia o objeto que faz a vetorização dos dados de texto:
vectorizer = CountVectorizer(analyzer="word", tokenizer=tokenizador)

In [15]:
# Aplica o vetorizador nos dados de texto e retorna uma matriz esparsa ( contendo vários zeros):
freq_textos = vectorizer.fit_transform(texto)
type(freq_textos)

scipy.sparse.csr.csr_matrix

In [16]:
# Treino de modelo de Machine Learning:
modelo = MultinomialNB()
modelo.fit(freq_textos,classes)

MultinomialNB()

In [17]:
testes = ['J conseguiu ser pior que o firmino',
          'Sandra Bullock é impecável em tudo o que faz',
          'Que zagueiraço é Van Dijk!! Quem fala que ele só teve uma temporada não sabe NADA de futebol. Já chegaram a compará-lo com Kimpembe, Rudiger, Varane etc. Não tem comparação!! Depois de Sérgio Ramos, Van Dijk é DISPARADO o melhor do mundo!',
          'Obg Atalanta por fazer eu perder 3 mil time de fdp do crl tô cheio de ódio',
          'florence linda te amo fada mãe']

testes = [Preprocessing(i) for i in testes]

In [18]:
freq_testes = vectorizer.transform(testes)

In [19]:
freq_testes

<5x160379 sparse matrix of type '<class 'numpy.int64'>'
	with 39 stored elements in Compressed Sparse Row format>

In [20]:
# 1= negativo
# 0= positivo
for t, c in zip (testes,modelo.predict(freq_testes)):
    print (t)
    print (c)

j conseguiu ser pior firmino
1
sandra bullock impecável tudo faz
0
zagueiraço van dijk!! fala temporada sabe nada futebol chegaram comparálo kimpembe, rudiger, varane etc comparação!! sérgio ramos, van dijk disparado melhor mundo!
0
obg atalanta fazer perder 3 mil time fdp crl tô cheio ódio
1
florence linda amo fada mãe
0


In [21]:
#probabilidade que o modelo fez de cada frase
print (modelo.classes_)
modelo.predict_proba(freq_testes).round(2)

[0 1]


array([[0.07, 0.93],
       [0.85, 0.15],
       [0.82, 0.18],
       [0.36, 0.64],
       [0.86, 0.14]])

In [33]:
df_teste=pd.read_csv('dados_testes.csv', encoding='utf-8')

In [34]:
temas = df_teste['tema'].value_counts()
dic_temas  = dict(temas)
nome_temas = list(dic_temas.keys())

In [37]:
df_de_um_tema = df_teste.query('tema == "Bruno Henrique"')

In [40]:
tweets = df_de_um_tema['texto']
tweets = [Preprocessing(i) for i in tweets]
tweets_testes = vectorizer.transform(tweets)
# 1= negativo
# 0= positivo
positivo=0
negativo=0
for t, c in zip (tweets,modelo.predict(tweets_testes)):
    if(c==1):
        negativo+=1
    else:
        positivo+=1
        
if(negativo>positivo):
    print("Negativo")
else:
    print("Positivo")

Positivo


In [62]:
tweets = df_teste['texto']
tweets = [Preprocessing(i) for i in tweets]
tweets_testes = vectorizer.transform(tweets)
# 1= negativo
# 0= positivo
for t, c in zip (tweets,modelo.predict(tweets_testes)):
    print (t)
    print (c)

negócio impressionante diego alves bruno henrique nasceram pra jogar jogo gigante
0
bruno henrique sentiu muscular fodeu
0
bruno henrique respira jogos decisivos impressionante
0
negócio impressionante diego alves bruno henrique nasceram pra jogar jogo gigante
0
sabia bruno henrique sentido senhor pesadelo
0
bruno henrique
0
bruno henrique levanta plmds agora
0
crlh bruno henrique ??? tmnc zica fudida 🤦 ‍♂️
1
bruno henrique amo
0
negócio impressionante diego alves bruno henrique nasceram pra jogar jogo gigante
0
bruno henrique hoje melhor jogador mundo sei amanhã, hoje certeza!
0
bruno henrique levanta faz cmg
0
sugiro bruno henrique engravide mulheres
0
foda desse flamengo jogo decisivo sempre mesmos aparecem olha bruno henrique diego alves jogand…
0
bruno henrique monstruoso decisivo demais!
0
bruno henrique fez gol libertadores cartola escalar time
0
tiro 12 bruno henrique joga frente
0
bruno henrique sentiu, ah
0
rei américa tá on, esqueceeee, bruno henrique pra caralho
0
levanta b

ajude lar maria lourdes! qualquer ajuda válida doar esqueça enviar comprovante dm faça…
0
judolffos lar
0
julho judolffos lar
0
gente, puder doar 1 real, doe! lar maria lourdes precisa! pix 02752088000123 judolffos lar
0
n judolffos lar
0
último desafio alfabeto z todos meses ano todos signos 4 estações ano 7 dias da…
0
junho judolffos lar
0
k judolffos lar
0
maio judolffos lar
0
g judolffos lar
0
março judolffos lar
0
judolffos lar
0
fevereiro judolffos lar
0
janeiro judolffos lar
0
último desafio alfabeto z todos meses ano todos signos 4 estações ano 7 dias da…
0
sagitário judolffos lar
0
escorpião judolffos lar
0
ajude lar maria lourdes! qualquer ajuda válida doar esqueça enviar comprovante dm faça…
0
câncer judolffos lar
0
virgem judolffos lar
0
leão judolffos lar
0
gêmeos judolffos lar
0
touro judolffos lar
0
áries judolffos lar
0
dezembro judolffos lar
0
novembro judolffos lar
0
judolffos lar
0
outubro judolffos lar
0
n judolffos lar
0
setembro judolffos lar
0
agosto judolffos la

bruno henrique pai barça fake
0
pai barça
0
conheço gente torce pra cr7fc falando barça
1
oi barça
0
realidade moradora ilha bebe umas pega barca sentindo jack sparrow
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
bruno henrique craque dms monstro jogador renight pai barça falso
0
ainda acho q vai ser 5x2 barça
0
oi barça
0
brasil, hoje 70 mil cantando nome jorge jesus, após benfica 3x0 barça
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
lyon psg têm base melhor barça vendem jogadores barça
0
jogadores barca falando q so 2 kkkk
0
oi barça
0
oi barça
0
bruno henrique fez 4 barça kkkkkkkkkkkkkkkkkkk
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
simplesmente bh acabou barça po
1
oi barça
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
oi barça
0
barça falso amor
0
bruno henrique fez um, poxa triste barça
0
@rafisk_215 messi saiu equador, barça
0
pra mim chega acaba jogo aí barça tá sendo humilhado
1
bh carrasco barça
0
oi barça
0
oi bar

po, baldasso bebeto maior judas história flamengo kkkkk bruno henrique 11 ideal da…
0
bruno henrique maior bebeto história flamengo boa noite todos!
0
bebeto??
1
adianta, bebeto sempre meteu muitos gols também, romário romário igualmente gabigol aparece mídia, polêmico daqui uns anos vão lembrar dois, vão lembrar dupla n vai lembrar mesma proporção
0
bruno henrique maior bebeto história flamengo boa noite todos!
0
sim, abriu mão fundão eleitoral antes falar besteira, legal vc pesquisar coisas boas tá fazendo
1
@y4n2204 acredito óbvio pra 98% flamenguistas, 2% sendo saudosistas bebeto campeão 87 jogando pra caralho aqui, traiu indo vasco estrelado (início eurico assumindo vascaíno
1
bruno henrique maior bebeto história flamengo boa noite todos!
0
bruno henrique maior bebeto história flamengo boa noite todos!
0
bruno henrique maior bebeto história flamengo boa noite todos!
0
hernane brocador ídolo doq bebeto
0
bruno henrique maior bebeto história flamengo boa noite todos!
0
vi bebeto jog